http://codekata.com/kata/kata09-back-to-the-checkout/

<h1>Kata09: Back to the Checkout</h1>


Back to the supermarket. This week, we’ll implement the code for a checkout system that handles pricing schemes such as “apples cost 50 cents, three apples cost $1.30.”

Way back in KataOne we thought about how to model the various options for supermarket pricing. We looked at things such as “three for a dollar,” “$1.99 per pound,” and “buy two, get one free.”

This week, let’s implement the code for a supermarket checkout that calculates the total price of a number of items. In a normal supermarket, things are identified using Stock Keeping Units, or SKUs. In our store, we’ll use individual letters of the alphabet (A, B, C, and so on). Our goods are priced individually. In addition, some items are multipriced: buy n of them, and they’ll cost you y cents. For example, item ‘A’ might cost 50 cents individually, but this week we have a special offer: buy three ‘A’s and they’ll cost you $1.30. In fact this week’s prices are:

~~~
  Item   Unit      Special
         Price     Price
  --------------------------
    A     50       3 for 130
    B     30       2 for 45
    C     20
    D     15

~~~
Our checkout accepts items in any order, so that if we scan a B, an A, and another B, we’ll recognize the two B’s and price them at 45 (for a total price so far of 95). Because the pricing changes frequently, we need to be able to pass in a set of pricing rules each time we start handling a checkout transaction.

The interface to the checkout should look like:

~~~

co = CheckOut.new(pricing_rules)
co.scan(item)
co.scan(item)
    :    :
price = co.total

~~~
There are lots of ways of implementing this kind of algorithm; if you have time, experiment with several.

<h2>Objectives of the Kata</h2>

To some extent, this is just a fun little problem. But underneath the covers, it’s a stealth exercise in decoupling. The challenge description doesn’t mention the format of the pricing rules. How can these be specified in such a way that the checkout doesn’t know about particular items and their pricing strategies? How can we make the design flexible enough so that we can add new styles of pricing rule in the future?


In [1]:
import datetime

class SKU():
    """
    Stock Keeping Unit. The lowest level of stuff 
    """
    def __init__(self, description, manufacturer=None, material=None, 
                 size=None, color=None, packaging=None):
        self.description = description
        
        self.manufacturer = manufacturer        
        self.material = material
        self.size = size
        self.color = color
        self.packaging = packaging
        
    def __repr__(self):
        return "<%s {'description':%s, 'size': %s}>" % (type(self).__name__, self.description, self.size)


class PricePoint():
    """
    The normal selling price of a SKU.
    Removing date range. Create feb_price_list if that functionality is needed
    """
    def __init__(self, SKU, price):
        self.SKU = SKU
        self.price = price
        
    def __repr__(self):
        return "<%s {'SKU':%s, 'price': %.2f}>" % (
            type(self).__name__, 
            self.SKU, 
            self.price,
        )
    
class PriceList():
    """
    A list of PricePoints for various SKUs.
    
    ToDo: 
        Now that valid dates are removed, convert _prices to a set ???
    """
    def __init__(self):
        self._prices = [] # A list of PricePoints
        self._discountFormulas = {} # A dict of DiscountFormulas
    
    def addPricePoint(self, pricePoint):
        # ToDo - deal with overlapping date ranges better than max()
        self._prices.append(pricePoint)
        
    def addSKU(self, SKU, price):
        pp = PricePoint(SKU, price)
        self.addPricePoint(pp)
        
    def getPrice(self, SKU):
        """
        Returns the largest valid price.
        Throws a SKU Not Found exception.
        """
        
        # Time for a lesson in list comprehension scopes :( 
        # Using lambda trick to get argument SKU into the list comprehension's scope
        # http://stackoverflow.com/a/28130950
        matchingPricePoints = (lambda SKU=SKU: [p for p in self._prices if p.SKU is SKU ])()
        
        #if len(matchingPricePoints) < 1:
        if matchingPricePoints is None:
            raise Exception('Price not found.')
            
        return sorted(matchingPricePoints, key=lambda pp: pp.price, reverse=True)[0].price
    
    def addDiscountFormula(self, discountFormula):
        self._discountFormulas[discountFormula.SKU] = discountFormula
        
    def getDiscountFormula(self, sku):
        return self._discountFormulas.get(sku, None)

        

class DiscountFormula():
    """
    How much to knock off the price. 
    Factor is the percentage off & should normally be negative ( but could be positive for punitive pricing )
    """
    def __init__(self, SKU, description, factor=-0.0):
        self.SKU = SKU
        self.description = description
        self.factor = factor
        
    def __str__(self):
        return self.description
        
    def value(self, quantity, unitPrice ):
        return quantity * unitPrice * self.factor
        

class DiscountFormula_m_for_n(DiscountFormula):
    """
    How much to knock off the price for a typical 3 for 2 special -aka- buy 2 get 1 free.
    Factor should normally be -1.0
    Use this formula when the price is an m for n ratio
    """
    def __init__(self, SKU, mfor, nfor, description, factor=-1.0):
        assert mfor > nfor # Can't be == or we'll get div 0 errors
        self.mfor = mfor
        self.nfor = nfor
        super().__init__(SKU, description, factor) 
        
    def value(self, quantity, unitPrice):
        if quantity >= self.mfor:
            return int( (self.mfor - self.nfor) / self.mfor * quantity) * unitPrice * self.factor
        else:
            return 0.0

class DiscountFormula_m_for_fixed(DiscountFormula):
    """
    How much to knock off the price for a 3 for $fixed special 
    Factor should normally be -1.0
    Use this formula if it's qty m for fixed price $fixed, rather than 3 for 2, 
    factor=-1
    """
    def __init__(self, SKU, mfor, specialPrice, description, factor=-1.0):
        assert mfor > 0
        self.mfor = mfor
        self.specialPrice = specialPrice
        super().__init__(SKU, description, factor) 
        
    def value(self, quantity, unitPrice):
        if quantity >= self.mfor:
            # discard fraction of unit
            return ((unitPrice  * int( quantity/self.mfor) * self.mfor) - \
                    (self.specialPrice * int( quantity/self.mfor))
                   ) * self.factor
        else:
            return 0.0

class DiscountLine():
    """
    Line item on a PurchaseOrder
    """
    def __init__(self, SKU, quantity, unitPrice, discountFormula):
        self.SKU = SKU
        self.discountFormula = discountFormula
        self.quantity = quantity
        self.unitPrice = unitPrice
        
    def __str__(self):
        return "%s -> %s @ %.2f" % (self.discountFormula.description, self.SKU.description, self.value())
    
    def value(self):
        return self.discountFormula.value(self.quantity, self.unitPrice )
        
    
class OrderLine():
    """
    Line item on a PurchaseOrder
    """
    def __init__(self, SKU, quantity, unitPrice):
        self.SKU = SKU
        self.quantity = quantity
        self.unitPrice = unitPrice
        
    def __str__(self):
        return "%d * %s @ %.2f" % (self.quantity, self.SKU.description, self.unitPrice)
        
    def value(self):
        """Quantity * Price"""
        return (self.quantity * self.unitPrice)
    
    
class PurchaseOrder():
    """
    List of OrderLines and DiscountLines
    Specify a priceList to use when instantiating.
    """
    def __init__(self, priceList):
        self._priceList = priceList
        self.orderLines = []
        self.discountLines = []
        
    def __str__(self):
        self.calcDiscountLines()
        return "\n".join([str(ol) for ol in self.orderLines])    + \
            "\nDiscounts \n"                                     + \
            "\n".join([str(dl) for dl in self.discountLines])    + \
            "\n------------------------------------------------" + \
            "\n               Total %.2f" % self.totalValue()    + \
            "\n================================================"
            
            
    def addOrderLine(self, SKU, quantity=1):
        ol = OrderLine(SKU, quantity, self._priceList.getPrice(SKU))
        self.orderLines.append(ol)
        
    def subtotalOrderLineValue(self, SKU):
        matchingOrderLines = (lambda SKU=SKU: [ol for ol in self.orderLines if ol.SKU is SKU])()
        return sum([ol.value() for ol in matchingOrderLines ])
    
    def subtotalOrderLineQuantity(self, SKU):
        matchingOrderLines = (lambda SKU=SKU: [ol for ol in self.orderLines if ol.SKU is SKU])()
        return sum([ol.quantity for ol in matchingOrderLines ])
                   
    def getSKUset(self):
        return set([ol.SKU for ol in self.orderLines])
            
    def calcDiscountLines(self):                   
        self.discountLines = [ \
                              DiscountLine(df.SKU, self.subtotalOrderLineQuantity(df.SKU), self._priceList.getPrice(df.SKU), df) \
                                for df in [self._priceList.getDiscountFormula(_sku) for _sku in self.getSKUset() ] \
                              if df is not None]
        self.discountLines = [dl for dl in self.discountLines if dl.value() != 0.0] 
       
    def totalValue(self):
        self.calcDiscountLines()
        return sum([ol.value() for ol in self.orderLines]) + sum([dl.value() for dl in self.discountLines])
    
    


<h2>ToDo</h2>
 * Send a purchaseOrder to Inventory for fulfillment
 * Draw down stock
 * Deal with insufficient stock 
 * Test drawdown ordering


<h2>Unit Testing</h2>

~~~
  Item   Unit      Special
         Price     Price
  --------------------------
    A     50       3 for 130
    B     30       2 for 45
    C     20
    D     15
~~~

Here’s a set of unit tests for a Ruby implementation. The helper method price lets you specify a sequence of items using a string, calling the checkout’s scan method on each item in turn before finally returning the total price.

~~~	
class TestPrice < Test::Unit::TestCase

  def price(goods)
    co = CheckOut.new(RULES)
    goods.split(//).each { |item| co.scan(item) }
    co.total
  end

  def test_totals
    assert_equal(  0, price(""))
    assert_equal( 50, price("A"))
    assert_equal( 80, price("AB"))
    assert_equal(115, price("CDBA"))

    assert_equal(100, price("AA"))
    assert_equal(130, price("AAA"))
    assert_equal(180, price("AAAA"))
    assert_equal(230, price("AAAAA"))
    assert_equal(260, price("AAAAAA"))

    assert_equal(160, price("AAAB"))
    assert_equal(175, price("AAABB"))
    assert_equal(190, price("AAABBD"))
    assert_equal(190, price("DABABA"))
  end

  def test_incremental
    co = CheckOut.new(RULES)
    assert_equal(  0, co.total)
    co.scan("A");  assert_equal( 50, co.total)
    co.scan("B");  assert_equal( 80, co.total)
    co.scan("A");  assert_equal(130, co.total)
    co.scan("A");  assert_equal(160, co.total)
    co.scan("B");  assert_equal(175, co.total)
  end
end
~~~


In [4]:
from unittest import *

class CheckoutTests(TestCase):
    
    @classmethod
    def setUpClass(self):
        pass
        
    def setUp(self):
        self.A = SKU( description="Tin baked beans in tomato sauce", manufacturer="Heinz")
        self.B = SKU( description="Tin cannolli beans in brine", manufacturer="Koo")
        self.C = SKU( description="Tin kidney beans", manufacturer="Heinz")
        self.D = SKU( description="Tin mixed 3 beans", manufacturer="Koo")

        self.retailPriceList = PriceList()        
        self.retailPriceList.addSKU(self.A, 50)
        self.retailPriceList.addSKU(self.B, 30)
        self.retailPriceList.addSKU(self.C, 20)
        self.retailPriceList.addSKU(self.D, 15)

        self.threeFor130Discount = DiscountFormula_m_for_fixed( self.A, 3, 130.0, "Special 3 A for 130" )
        self.retailPriceList.addDiscountFormula(self.threeFor130Discount)

        self.twoFor45Discount    = DiscountFormula_m_for_fixed( self.B, 2, 45.0,  "Special 2 B for 45"  )
        self.retailPriceList.addDiscountFormula(self.twoFor45Discount)
    
    def test_full_workflow(self):
        """
        Create a checkout and process an order
        """
        checkout = PurchaseOrder(self.retailPriceList)
        checkout.addOrderLine(self.A)
        self.assertEqual(checkout.totalValue(), 50.0)

        checkout.addOrderLine(self.B)
        self.assertEqual(checkout.totalValue(), 80.0)

        checkout.addOrderLine(self.A, 1)
        self.assertEqual(checkout.totalValue(), 130.0)

        checkout.addOrderLine(self.A, 1)
        self.assertEqual(checkout.totalValue(), 160.0)

        checkout.addOrderLine(self.B, 1)
        self.assertEqual(checkout.totalValue(), 175.0)

        #print(checkout)
        
    def _price(self, itemList):
        co = PurchaseOrder(self.retailPriceList)
        for x in itemList:
            co.addOrderLine(x)  
            
        #print(co)
        return co.totalValue()

        
    def test_bulk(self):
        A = self.A
        B = self.B
        C = self.C
        D = self.D 
        
        self.assertEqual(  0, self._price([]))
        self.assertEqual( 50, self._price([A]))
        self.assertEqual( 80, self._price([A, B]))
        self.assertEqual(115, self._price([C, D, B, A]))
        
        self.assertEqual(100, self._price([A,A]))        
        self.assertEqual(130, self._price([A,A,A]))
        self.assertEqual(180, self._price([A,A,A,A]))
        self.assertEqual(230, self._price([A,A,A,A,A]))
        self.assertEqual(260, self._price([A,A,A,A,A,A]))
        
        self.assertEqual(160, self._price([A,A,A,B]))
        self.assertEqual(175, self._price([A,A,A,B,B]))
        self.assertEqual(190, self._price([A,A,A,B,B,D]))
        self.assertEqual(190, self._price([D,A,B,A,B,A]))


cot = CheckoutTests()

suite = TestLoader().loadTestsFromModule(cot)
TextTestRunner().run(suite)

..
----------------------------------------------------------------------
Ran 2 tests in 0.006s

OK


<unittest.runner.TextTestResult run=2 errors=0 failures=0>